In [ ]:
from pathlib import Path
import open3d as o3d
from typing import NamedTuple
import plotly.graph_objects as go
import numpy as np

C0 = 0.28209479177387814
def SH2RGB(sh):
    return sh * C0 + 0.5

class BasicPointCloud(NamedTuple):
    points : np.array
    colors : np.array
    normals : np.array


class NoisyGrow():
    class Config():
        radius: float = 4
        sh_degree: int = 0
        load_type: int = 0
        load_path: str = "./load/shapes/stand.obj"

    def __init__(self, data_dir, threshold, density):
        if isinstance(data_dir, str):
            data_dir = Path(data_dir)
        self.path = data_dir
        self.radius = 4.0
        self.threshold = threshold
        self.density = density
        with open(self.path / "caption.txt") as f:
            self.caption = f.read()
        self.pcb()

    def add_points(self,coords,rgb):
        self.pcd_by3d = o3d.geometry.PointCloud()
        self.pcd_by3d.points = o3d.utility.Vector3dVector(np.array(coords))
        self.pcd_by3d.colors = o3d.utility.Vector3dVector(np.array(rgb))

        bbox = self.pcd_by3d.get_axis_aligned_bounding_box()
        volumn = bbox.volume()  # equivalent to np.abs(bbox.min_bound - bbox.max_bound).cumprod()[-1]
        np.random.seed(0)
        points = np.random.uniform(low=np.asarray(bbox.min_bound), high=np.asarray(bbox.max_bound), size=(int(self.density * volumn), 3))

        kdtree = o3d.geometry.KDTreeFlann(self.pcd_by3d)

        points_inside = []
        color_inside= []
        for point in points:
            _, idx, _ = kdtree.search_knn_vector_3d(point, 1)
            nearest_point = np.asarray(self.pcd_by3d.points)[idx[0]]
            if np.linalg.norm(point - nearest_point) < self.threshold:
                points_inside.append(point)
                color_inside.append(rgb[idx[0]]+0.2*np.random.random(3))

        all_coords = np.array(points_inside)
        all_rgb = np.array(color_inside)
        all_coords = np.concatenate([all_coords,coords],axis=0)
        all_rgb = np.concatenate([all_rgb,rgb],axis=0)
        return all_coords, all_rgb

    def smpl(self):
        self.num_pts  = 50000
        mesh = o3d.io.read_triangle_mesh(self.load_path)
        point_cloud = mesh.sample_points_uniformly(number_of_points=self.num_pts)
        coords = np.array(point_cloud.points)
        shs = np.random.random((self.num_pts, 3)) / 255.0
        rgb = SH2RGB(shs)
        adjusment = np.zeros_like(coords)
        adjusment[:,0] = coords[:,2]
        adjusment[:,1] = coords[:,0]
        adjusment[:,2] = coords[:,1]
        current_center = np.mean(adjusment, axis=0)
        center_offset = -current_center
        adjusment += center_offset
        return adjusment,rgb,0.5

    def pcb(self):
        data = np.load(self.path / "8192.npy")
        coords = data[:, :3]
        rgb = data[:, 3:]

        scale = 0.45

        bound= self.radius*scale
        all_coords,all_rgb = self.add_points(coords,rgb)

        pcd = BasicPointCloud(points=all_coords *bound, colors=all_rgb, normals=np.zeros((all_coords.shape[0], 3)))
        self.grown_pcd = pcd
        # plot_3d2(pcd.points, pcd.colors)

        return pcd
    
    def __call__(self):
        self.pcb()

    def plot_3d(self, original=False):
        if original:
            xyz = np.asarray(self.pcd_by3d.points)
            rgb = np.asarray(self.pcd_by3d.colors)
        else:
            xyz = self.grown_pcd.points
            rgb = self.grown_pcd.colors
        print("num points:", xyz.shape[0])

        # Normalize RGB values to [0, 1] range and prepare for Plotly
        rgb = rgb / 255.0  # Normalize RGB
        color_str = ['rgba({},{},{},{})'.format(int(r[0]*255), int(r[1]*255), int(r[2]*255), 0.6) for r in rgb]

        # Create a Plotly Graph object for the 3D scatter plot
        fig = go.Figure(data=[go.Scatter3d(
            x=xyz[:, 0],  # X coordinates
            y=xyz[:, 1],  # Y coordinates
            z=xyz[:, 2],  # Z coordinates
            mode='markers',
            marker=dict(
                size=2,
                color=color_str,  # Set color to the RGBA values
                opacity=0.6
            )
        )])

        # Update plot layout
        fig.update_layout(
            title={
                'text': self.caption,
                'font': {'color': 'white'}
            },
            paper_bgcolor='rgb(0,0,0)',  # White background
            plot_bgcolor='rgb(0,0,0)',  # White background
            scene=dict(
                xaxis=dict(showticklabels=False, title_text='', showgrid=False, zeroline=False, backgroundcolor='rgb(255,255,255)'),  # X axis settings
                yaxis=dict(showticklabels=False, title_text='', showgrid=False, zeroline=False, backgroundcolor='rgb(255,255,255)'),  # Y axis settings
                zaxis=dict(showticklabels=False, title_text='', showgrid=False, zeroline=False, backgroundcolor='rgb(255,255,255)'),  # Z axis settings
                aspectmode='auto'  # How aspect ratio is handled
            ),
            width=800,
            height=600,
            margin=dict(l=0, r=0, b=0, t=50)
        )

        # Show the plot
        fig.show()


In [2]:
from threestudio.systems.GaussianDreamer import GaussianDreamer
from gaussiansplatting.scene.gaussian_model import GaussianModel
from threestudio.data.uncond import RandomCameraDataModule, RandomCameraDataset, RandomCameraDataModuleConfig


point_cloud = o3d.io.read_point_cloud("..../groundtruth.ply")
colors = np.asarray(point_cloud.colors)
points = np.asarray(point_cloud.points)
normals = np.asarray(point_cloud.normals)

pcd = BasicPointCloud(points, colors, normals)


dreamer = GaussianDreamer()
dreamer.configure()
dreamer.gaussian.create_from_pcd(pcd)
###### or #####
# dreamer.gaussian.load_ply("....saved gaussian .ply file")

loader = RandomCameraDataModule().val_dataloader()
for batch in loader:
    images = dreamer.forward(batch)['comp_rgb']
    # visualize / save images here

ModuleNotFoundError: No module named 'tinycudann'